**The purpose of this notebook is to gather the web-scraped csv's and merge them into one dataframe that will be used for linear regression**

In [2]:
import pandas as pd
import glob
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']  # or svg
%matplotlib inline

sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [3]:
nba_box = pd.concat(
    map(pd.read_csv, ['data/nba_box_2018.csv', 'data/nba_box_2019.csv',
                     'data/nba_box_2020.csv','data/nba_box_2021.csv']), ignore_index=True)

nba_box

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,YEAR
0,Álex Abrines,SG,24,OKC,75,8,15.1,1.5,3.9,0.395,...,0.3,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7,2018
1,Quincy Acy,PF,27,BRK,70,8,19.4,1.9,5.2,0.356,...,0.6,3.1,3.7,0.8,0.5,0.4,0.9,2.1,5.9,2018
2,Steven Adams,C,24,OKC,76,76,32.7,5.9,9.4,0.629,...,5.1,4.0,9.0,1.2,1.2,1.0,1.7,2.8,13.9,2018
3,Bam Adebayo,C,20,MIA,69,19,19.8,2.5,4.9,0.512,...,1.7,3.8,5.5,1.5,0.5,0.6,1.0,2.0,6.9,2018
4,Arron Afflalo,SG,32,ORL,53,3,12.9,1.2,3.1,0.401,...,0.1,1.2,1.2,0.6,0.1,0.2,0.4,1.1,3.4,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,0.462,...,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0,2021
2724,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,0.559,...,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1,2021
2725,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,0.438,...,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3,2021
2726,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,0.559,...,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4,2021


In [4]:
nba_per100 = pd.concat(
    map(pd.read_csv, ['data/nba_per100_2018.csv', 'data/nba_per100_2019.csv',
                     'data/nba_per100_2020.csv','data/nba_per100_2021.csv']), ignore_index=True)
nba_per100

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,YEAR
0,Álex Abrines,SG,24,OKC,75,8,1134,5.0,12.7,0.395,...,5.0,1.2,1.7,0.4,1.1,5.4,15.4,116.0,110,2018
1,Quincy Acy,PF,27,BRK,70,8,1359,4.6,13.0,0.356,...,9.2,2.0,1.2,1.0,2.1,5.3,14.7,99.0,110,2018
2,Steven Adams,C,24,OKC,76,76,2487,8.9,14.2,0.629,...,13.7,1.8,1.8,1.6,2.6,4.3,21.1,125.0,107,2018
3,Bam Adebayo,C,20,MIA,69,19,1368,6.4,12.5,0.512,...,14.0,3.7,1.2,1.5,2.4,5.1,17.5,116.0,105,2018
4,Arron Afflalo,SG,32,ORL,53,3,682,4.7,11.6,0.401,...,4.7,2.2,0.3,0.6,1.5,4.0,12.8,98.0,115,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,Delon Wright,PG,28,SAC,27,8,696,7.2,15.5,0.462,...,7.2,6.7,3.0,0.8,2.4,2.1,18.7,115.0,115,2021
2724,Thaddeus Young,PF,32,CHI,68,23,1652,10.9,19.4,0.559,...,12.4,8.5,2.2,1.2,4.0,4.5,24.2,116.0,110,2021
2725,Trae Young,PG,22,ATL,63,63,2125,11.3,25.7,0.438,...,5.7,13.8,1.2,0.3,6.0,2.6,36.9,117.0,116,2021
2726,Cody Zeller,C,28,CHO,48,21,1005,8.8,15.7,0.559,...,15.9,4.2,1.3,0.8,2.5,5.9,21.9,123.0,112,2021


In [7]:
nba_per100_small = nba_per100[['ORtg','DRtg']]
nba_per100_small.head()

,ORtg,DRtg
0,116.0,110
1,99.0,110
2,125.0,107
3,116.0,105
4,98.0,115


In [8]:
nba_stats = nba_box.merge(nba_per100_small, how='inner', left_index=True, right_index=True)
nba_stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,YEAR,ORtg,DRtg
0,Álex Abrines,SG,24,OKC,75,8,15.1,1.5,3.9,0.395,...,1.5,0.4,0.5,0.1,0.3,1.7,4.7,2018,116.0,110
1,Quincy Acy,PF,27,BRK,70,8,19.4,1.9,5.2,0.356,...,3.7,0.8,0.5,0.4,0.9,2.1,5.9,2018,99.0,110
2,Steven Adams,C,24,OKC,76,76,32.7,5.9,9.4,0.629,...,9.0,1.2,1.2,1.0,1.7,2.8,13.9,2018,125.0,107
3,Bam Adebayo,C,20,MIA,69,19,19.8,2.5,4.9,0.512,...,5.5,1.5,0.5,0.6,1.0,2.0,6.9,2018,116.0,105
4,Arron Afflalo,SG,32,ORL,53,3,12.9,1.2,3.1,0.401,...,1.2,0.6,0.1,0.2,0.4,1.1,3.4,2018,98.0,115


In [10]:
nba_salary = pd.concat(
    map(pd.read_csv, ['data/nba_salary_2018.csv', 'data/nba_salary_2019.csv',
                     'data/nba_salary_2020.csv','data/nba_salary_2021.csv']), ignore_index=True)
nba_salary.head()

,RK,NAME,TEAM,SALARY,YEAR
0,1,Stephen Curry,Golden State Warriors,37457154,2018
1,2,Blake Griffin,Detroit Pistons,32088932,2018
2,3,Paul Millsap,Denver Nuggets,31269231,2018
3,4,Kyle Lowry,Toronto Raptors,31200000,2018
4,5,Gordon Hayward,Boston Celtics,29727900,2018


In [11]:
#Combine nba stats with respective salaries
nba_merge = pd.merge(nba_stats, nba_salary,  how='left', left_on=['Player','YEAR'], right_on = ['NAME','YEAR'])

In [12]:
#Take out players that did not get mapped
nba_df = nba_merge[nba_merge['SALARY'].notnull()]

#Remove redundant or extraneous columns
nba_df = nba_df.drop(columns=['RK','NAME','TEAM'])

In [13]:
nba_df.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'YEAR', 'ORtg',
       'DRtg', 'SALARY'],
      dtype='object')

In [20]:
#Obtain salary cap and scaling factor to get all Y values in 2021 terms
salary_cap = pd.read_csv('data/salary_cap.csv')
salary_cap['scale'] = salary_cap.iloc[36]['salary_cap'] / salary_cap['salary_cap']
salary_cap.tail()

,year,salary_cap,scale
32,2017,94143000.0,1.159300
33,2018,99093000.0,1.101390
34,2019,101869000.0,1.071376
35,2020,109140000.0,1.000000
36,2021,109140000.0,1.000000


In [21]:
#Merge total salary cap and 2021 scaling factor to main dataframe
nba_df = pd.merge(nba_df, salary_cap,  how='left', left_on=['YEAR'], right_on = ['year'])
nba_df = nba_df.drop(columns='year')

In [22]:
#Apply scaling factor to salary to express player salaries in 2021 dollars
nba_df['SALARY_2021'] = nba_df['SALARY'] * nba_df['scale']
nba_df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV,PF,PTS,YEAR,ORtg,DRtg,SALARY,salary_cap,scale,SALARY_2021
0,Steven Adams,C,24,OKC,76,76,32.7,5.9,9.4,0.629,...,1.7,2.8,13.9,2018,125.0,107,22471910.0,99093000.0,1.10139,2.475033e+07
1,Bam Adebayo,C,20,MIA,69,19,19.8,2.5,4.9,0.512,...,1.0,2.0,6.9,2018,116.0,105,2955840.0,99093000.0,1.10139,3.255531e+06
2,Cole Aldrich,C,29,MIN,21,0,2.3,0.2,0.7,0.333,...,0.0,0.5,0.6,2018,85.0,107,7300000.0,99093000.0,1.10139,8.040144e+06
3,LaMarcus Aldridge,C,32,SAS,75,75,33.5,9.2,18.0,0.510,...,1.5,2.1,23.1,2018,118.0,105,21461010.0,99093000.0,1.10139,2.363693e+07
4,Jarrett Allen,C,19,BRK,72,31,20.0,3.3,5.5,0.589,...,1.1,2.0,8.2,2018,119.0,108,2034120.0,99093000.0,1.10139,2.240359e+06


In [25]:
nba_df.isna().sum()

Player           0
Pos              0
Age              0
Tm               0
G                0
GS               0
MP               0
FG               0
FGA              0
FG%              5
3P               0
3PA              0
3P%            110
2P               0
2PA              0
2P%             18
eFG%             5
FT               0
FTA              0
FT%             71
ORB              0
DRB              0
TRB              0
AST              0
STL              0
BLK              0
TOV              0
PF               0
PTS              0
YEAR             0
ORtg             4
DRtg             0
SALARY           0
salary_cap       0
scale            0
SALARY_2021      0
dtype: int64

Most of the fields with nulls apply to shooting percentages. In other words, players that have not attempted a shot will have null percentages. I'm going to drop these values as it only applies to a small subset of players with very little playing time

In [26]:
nba_df.dropna(subset=['FT%','eFG%','ORtg'],inplace=True)

In [27]:
#Check for duplicates, there are none
nba_df[nba_df.duplicated() == True]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV,PF,PTS,YEAR,ORtg,DRtg,SALARY,salary_cap,scale,SALARY_2021


In [28]:
nba_df.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'YEAR', 'ORtg',
       'DRtg', 'SALARY', 'salary_cap', 'scale', 'SALARY_2021'],
      dtype='object')

In [30]:
#Save merged dataframe to csv to be used for regression
nba_df.to_csv('data/nba_df.csv',index=False)